In [9]:
import pandas as pd


# => 길이, 너비 둘다 언급될 경우, 30% 가중치 추가
# 5) 추천 규칙 적용 함수
#     (base = (L+A)/2, inter = min(L,A),
#      final = 0.7*base + 0.3*inter → 정규화)
# def combine(length_p, area_p, alpha=0.7, beta=0.3):


# 1) 파일 경로 (경로만 맞춰주면 됨)
length_path = "basic_data/네일 추천 분류 - 손톱 길이.csv"
area_path   = "basic_data/네일 추천 분류 - 손톱 면적.csv"

# 2) CSV 로드
def load_csv(path):
    for enc in ["utf-8-sig", "cp949", "utf-8"]:
        try:
            return pd.read_csv(path, encoding=enc)
        except:
            pass
    raise Exception(f"Failed to load CSV: {path}")

df_len = load_csv(length_path)
df_area = load_csv(area_path)

# 3) 전처리
# 유지할 5가지 네일 쉐입
shapes = ["square", "round", "oval", "squoval", "almond"]

# 길이 짧은/중간/긴 값만 남김
df_len = df_len[~df_len["길이"].isin(["보편적"])]

# 한글 → 영어 값 매핑
length_map = {
    "짧은": "short",
    "중간": "mid",
    "긴": "long"
}
area_map = {
    "넓은": "wide",
    "좁은": "narrow"
}
df_len["길이"] = df_len["길이"].map(length_map).fillna(df_len["길이"])
df_area["면적"] = df_area["면적"].map(area_map).fillna(df_area["면적"])

# 4) 길이/면적별 비율 계산 함수
def compute_freq(df, group_col):
    """
    df[group_col] 기준으로 5가지 쉐입만 필터한 비율(%) 계산
    """
    out = {}
    for cond, sub in df.groupby(group_col):
        cnt = sub["네일쉐입(네일팁)"].value_counts()

        # 5쉐입만 필터
        filtered = {s: cnt.get(s, 0) for s in shapes}
        total = sum(filtered.values())

        if total == 0:
            freq = {s: 0 for s in shapes}
        else:
            freq = {s: filtered[s] / total for s in shapes}

        out[cond] = freq
    return out

length_freq = compute_freq(df_len, "길이")
area_freq   = compute_freq(df_area, "면적")

# 5) 추천 규칙 적용 함수
#    (base = (L+A)/2, inter = min(L,A),
#     final = 0.7*base + 0.3*inter → 정규화)
def combine(length_p, area_p, alpha=0.7, beta=0.3):
    result = {}
    for s in shapes:
        L = length_p.get(s, 0)
        A = area_p.get(s, 0)

        base  = (L + A) / 2
        inter = min(L, A)
        final = alpha * base + beta * inter

        result[s] = final

    # 정규화
    total = sum(result.values())
    return {s: result[s] / total for s in shapes}

# 6) 길이 × 면적 모든 조합 계산
all_combos = {}

for length_cond, len_freq in length_freq.items():
    for area_cond, are_freq in area_freq.items():
        key = f"{length_cond}-{area_cond}"
        all_combos[key] = combine(len_freq, are_freq)

# 7) 결과를 DataFrame으로 변환
df_result = pd.DataFrame(all_combos).T
print(df_result)

# 8) 엑셀 저장
#df_result.to_excel("네일쉐입_추천비율_결과.xlsx")
#print("\n엑셀 저장 완료: 네일쉐입_추천비율_결과.xlsx")

                square     round      oval   squoval    almond
long-narrow   0.606952  0.037433  0.068627  0.149733  0.137255
long-wide     0.314815  0.064815  0.250000  0.000000  0.370370
mid-narrow    0.509470  0.039773  0.145833  0.159091  0.145833
mid-wide      0.236842  0.061404  0.350877  0.000000  0.350877
short-narrow  0.334775  0.361643  0.071340  0.208462  0.023780
short-wide    0.150052  0.389927  0.259605  0.021807  0.178609
